## Install the dependencies

In [ ]:
!pip install transformers datasets evaluate rouge_score

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset loading

In [ ]:
# load billsum dataset
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

In [ ]:
billsum = billsum.train_test_split(test_size=0.2)

In [ ]:
billsum['train'][0]

{'text': 'The people of the State of California do enact as follows:\n\n\nSECTION 1.\nSection 18120 of the Penal Code is amended to read:\n18120.\n(a) A person subject to a gun violence restraining order issued pursuant to this division shall not have in his or her custody or control, own, purchase, possess, or receive any firearms or ammunition while that order is in effect.\n(b) (1) Upon issuance of a gun violence restraining order issued pursuant to this division, the court shall order the restrained person to surrender all firearms and ammunition in the restrained person’s custody or control, or which the restrained person possesses or owns pursuant to paragraph (2).\n(2) The surrender ordered pursuant to paragraph (1) shall occur by immediately surrendering all firearms and ammunition in a safe manner, upon request of any law enforcement officer, to the control of the officer, after being served with the restraining order. A law enforcement officer serving a gun violence restraini

## preprocessing

In [ ]:
from transformers import AutoTokenizer

In [ ]:
checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

## for dynamic padding

In [ ]:
# for padding dynamic
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

## evaluation matrix

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    print('results of rouge: {}'.format(result.items()))

    return {k: round(v, 4) for k, v in result.items()}

## Train

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [ ]:
# !pip install transformers[torch]
!pip install accelerate -U

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="my_first_t5_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.617571,0.131400,0.041200,0.108700,0.108700,19.000000
2,No log,2.528976,0.141200,0.053800,0.119700,0.119900,19.000000
3,No log,2.476114,0.180200,0.086600,0.154100,0.154300,19.000000
4,No log,2.441154,0.192400,0.097600,0.164200,0.164600,19.000000
5,No log,2.417815,0.194900,0.099600,0.166900,0.167100,19.000000
6,No log,2.400876,0.195500,0.100100,0.167400,0.167800,19.000000
7,No log,2.388021,0.194900,0.099400,0.166800,0.167000,19.000000
8,No log,2.379890,0.196000,0.100000,0.167500,0.167800,19.000000
9,2.588900,2.374719,0.196700,0.100000,0.168200,0.168400,19.000000
10,2.588900,2.372993,0.195700,0.099700,0.167300,0.167600,19.000000


TrainOutput(global_step=620, training_loss=2.5650133194461944, metrics={'train_runtime': 725.5992, 'train_samples_per_second': 13.63, 'train_steps_per_second': 0.854, 'total_flos': 2677060833116160.0, 'train_loss': 2.5650133194461944, 'epoch': 10.0})

## Inference

In [ ]:
from transformers import pipeline

text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
summarizer = pipeline("summarization", model="/content/my_first_t5_billsum_model")
summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country."}]

In [ ]:
from transformers import AutoTokenizer

text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
tokenizer = AutoTokenizer.from_pretrained("/content/my_first_t5_billsum_model")
inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("/content/my_first_t5_billsum_model")
outputs = model.generate(inputs, max_new_tokens=20, do_sample=False)

In [ ]:
outputs

tensor([[   0,   37,   86,   89, 6105,  419, 8291, 1983, 1364,    7, 7744, 2672,
         1358,    6,  533,  124, 1358,    6,   11,  827, 1358]])

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs'

## Evaluation

In [ ]:
# Model evaluation using
def compute_metrics_str(predictions, labels):
    # predictions, labels = eval_pred
    # decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=predictions, references=labels, use_stemmer=True)

    # prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    # result["gen_len"] = np.mean(prediction_lens)
    print('results of rouge: {}'.format(result.items()))

    return {k: round(v, 4) for k, v in result.items()}, result

accuracy_matrix, results = compute_metrics_str(['again kerala'],['again kerala'])

In [ ]:
accuracy_matrix

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

In [ ]:
results

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}

## onnx conversion

In [ ]:
# load model and tokenizer
model_id = "/content/my_first_t5_billsum_model"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
dummy_model_input = tokenizer("This is a sample", return_tensors="pt")

In [ ]:
from pathlib import Path
import transformers
from transformers.onnx import FeaturesManager
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM

# load model and tokenizer
model_id = "/content/my_first_t5_billsum_model"
feature = "seq2seq-lm"
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# load config
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
onnx_config = model_onnx_config(model.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
        preprocessor=tokenizer,
        model=model,
        config=onnx_config,
        opset=13,
        output=Path("/content/my_first_t5_billsum_model/t5-model.onnx")
)

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



## onnx inference

In [ ]:
from onnxruntime import InferenceSession

session = InferenceSession("/content/my_first_t5_billsum_model/t5-model.onnx", providers=['CUDAExecutionProvider', 'AzureExecutionProvider', 'CPUExecutionProvider'])

/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [ ]:
model_id = "/content/my_first_t5_billsum_model"

text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."
tokenizer = AutoTokenizer.from_pretrained("/content/my_first_t5_billsum_model")

model_input = tokenizer(text, return_tensors="np")



In [ ]:
model_input['decoder_input_ids'] = model_input['input_ids']
model_input['decoder_attention_mask'] = model_input['attention_mask']

In [ ]:
outputs = session.run(input_feed=model_input, output_names="last_hidden_state")

TypeError: ignored